# DATA 620 - Final Project

Jeremy OBrien, Mael Illien, Vanita Thompson

## Topic Networks

### Introduction

A powerful analytical application of NLP is topic modeling, which identifies the themes present in a corpus comprised of multiple documents based on the words in those documents. Because it can be used to uncover the thematic structure in documents, topic modeling has an array of applications in information retrieval and document mining.

Topics are 'probability distributions over a fixed vocabulary'. An overview of topic modeling and its current applications in bioinformatics, and it's common to use probabilistic generative models derived from LDA (Latent Dirichlet Allocation) to model in an unsupervised fashion the latent semantic structure of documents. Topic models can be tuned and optimized in a variety of ways, including improving how topics are segregated from each other and calibrating for a useful number of topics.

### Research Question
We will combine techniques from topic modeling and network analysis to address this question.

Given a text corpus comprised of multiple documents, what are the topics of those documents and how are the documents thematically related to one another?

### Approach
- Leverage the Reuters news corpus of nearly 11,000 articles (labeled with at least one category each; unfortunately, authorship is not labeled)
- Using the NLTK, Spacy, and Gensim packages, implement and tune an unsupervised LDA-based topic model (i.e. without reference to the provided article topic labels)
- Analyze model perplexity and coherence, overall topic prevalence, and topic distribution across articles
- Generate a bipartite, weighted (likely on coherence) graph of articles and topics, and analyze its topology to identify relationships between topics

## Setup

In [1]:
import re
import random
import numpy as np
import pandas as pd
from pprint import pprint

np.random.seed(12321)

import nltk
from nltk.corpus import stopwords
from nltk.corpus import reuters

import spacy  # need to install

import gensim  # need to install
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
# warnings.filterwarnings('ignore', category=DeprecationWarning)

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

## Data Import & Processing

### Data import and preliminary EDA

In [2]:
# NLTK Reuters corpus test
from nltk.corpus import reuters
reuters.fileids()
reuters.words('test/14841')[-10:]

['the', 'shipment', 'was', 'for', 'April', '8', 'to', '20', 'delivery', '.']

Here are 25 sampled categoies from the Reuters corpus. These correspond to themes/topics assigned to documents in the corpus and are essentially labels for the articles.

In [3]:
# Remove
random.choices(reuters.categories(),k=25)

['oat',
 'gas',
 'palmkernel',
 'palladium',
 'rape-oil',
 'orange',
 'wheat',
 'nat-gas',
 'sorghum',
 'cpu',
 'groundnut-oil',
 'housing',
 'soybean',
 'corn',
 'ipi',
 'fuel',
 'rand',
 'barley',
 'jet',
 'palmkernel',
 'pet-chem',
 'orange',
 'soy-meal',
 'coffee',
 'money-fx']

In [4]:
# Adjust to generate dataframe directly from nltk corpus
# https://stackoverflow.com/questions/46109166/converting-categorizedplaintextcorpusreader-into-dataframe
news = []
for fileid in reuters.fileids():
    tag, filename = fileid.split('/')
    news.append((filename, tag, reuters.categories(fileid), reuters.raw(fileid)))

df = pd.DataFrame(news, columns=['filename', 'tag', 'categories','text'])

In [5]:
# hide tag column
df

,filename,tag,categories,text
0,14826,test,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,14828,test,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,14829,test,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,14832,test,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,14833,test,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...
...,...,...,...,...
10783,999,training,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
10784,9992,training,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...
10785,9993,training,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...
10786,9994,training,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...


In [6]:
# Output with print
# Describe shape of data

# 10,788 separate cases (news articles)
# df.shape
# len(df.filename.unique())

# Each case is tagged as either train or test
# df.tag.unique()

# There are 10,657 articles with distinct text body - so 131 duplicates
# len(df.text.unique())

# Check the text body of duplicates (except for first instance)
# print(df[df.text.duplicated('first')])

# JO: Evaluate cause and consider case for removal
# MI: I checked with some of the indexes and found no overlap. See below

In [7]:
#print(df[df.text.duplicated('first')])

### Data cleaning and deduplication

In [8]:
# Convert to list
data = df['text'].values.tolist()

# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]

pprint(data[330][:1000])

('G-7 ISSUES STATEMENT AFTER MEETING Following is the text of a statement by '
 'the Group of Seven -- the U.S., Japan, West Germany, France, Britain, Italy '
 'and Canada -- issued after a Washington meeting yesterday. 1. The finance '
 'ministers and central bank governors of seven major industrial countries met '
 'today. They continued the process of multilateral surveillance of their '
 'economies pursuant to the arrangements for strengthened economic policy '
 'coordination agreed at the 1986 Tokyo summit of their heads of state or '
 'government. The managing director of the International Monetary Fund also '
 'participated in the meeting. 2. The ministers and governors reaffirmed the '
 'commitment to the cooperative approach agreed at the recent Paris meeting, '
 'and noted the progress achieved in implementing the undertakings embodied in '
 'the Louvre Agreement. They agreed, however, that further actions will be '
 'essential to resist rising protectionist pressures, sustai

In [9]:
# only one example
pprint(data[333][:1000])

('JAPAN BUSINESS LEADERS SAY G-7 ACCORD IS WORRYING The leaders of two of '
 'Japans top business groups said in separate statements the Group of Seven '
 '(G-7) accord reached in Washington yesterday is of deep concern to Japan '
 'because it shows the major industrial nations regard the yens current level '
 'as appropriate. Eishiro Saito, chairman of the Federation of Economic '
 'Organizations (Keidanren), said the yens present rate is well above adequate '
 'levels. He did not elaborate. Takashi Ishihara, chairman of the Japan '
 'Committee for Economic Development, said the accord will not prevent the yen '
 'from rising further. "We do not understand why the G-7 approved present '
 'rates as the yen has risen excessively since the Paris accord," Ishihara '
 'said. G-7 members Britain, Canada, France, Italy, Japan, the U.S. And West '
 'Germany said in a statement they consider their currencies are now within '
 'ranges broadly consistent with economic fundamentals. Saito called 

### Text preparation and feature engineering

In [10]:
# Tokenize words and clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#truncate
print(data_words[0])

['asian', 'exporters', 'fear', 'damage', 'from', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asias', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen', 'and', 'officials', 'said', 'they', 'told', 'reuter', 'correspondents', 'in', 'asian', 'capitals', 'move', 'against', 'japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'and', 'lead', 'to', 'curbs', 'on', 'american', 'imports', 'of', 'their', 'products', 'but', 'some', 'exporters', 'said', 'that', 'while', 'the', 'conflict', 'would', 'hurt', 'them', 'in', 'the', 'long', 'run', 'in', 'the', 'short', 'term', 'tokyos', 'loss', 'might', 'be', 'their', 'gain', 'the', 'has', 'said', 'it', 'will', 'impose', 'mln', 'dlrs', 'of', 'tariffs', 'on', 'imports', 'of', 'japanese', 'electronics', 'goods', 'on', 'april', 'in', 'retaliation', 'for', 'japans', 'alleged', 'failur

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['asian', 'exporters', 'fear', 'damage', 'from', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asias', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen', 'and', 'officials', 'said', 'they', 'told', 'reuter_correspondents', 'in', 'asian', 'capitals', 'move', 'against', 'japan', 'might', 'boost', 'protectionist_sentiment', 'in', 'the', 'and', 'lead', 'to', 'curbs', 'on', 'american', 'imports', 'of', 'their', 'products', 'but', 'some', 'exporters', 'said', 'that', 'while', 'the', 'conflict', 'would', 'hurt', 'them', 'in', 'the', 'long', 'run', 'in', 'the', 'short_term', 'tokyos', 'loss', 'might', 'be', 'their', 'gain', 'the', 'has', 'said', 'it', 'will', 'impose', 'mln', 'dlrs', 'of', 'tariffs', 'on', 'imports', 'of', 'japanese', 'electronics', 'goods', 'on', 'april', 'in', 'retaliation', 'for', 'japans', 'alleged_failure', 'to', 's

### Pre-processing

In [12]:
# Set up stopwords
stop_words = stopwords.words('english')

# JO: Confirm if these / other stopwords should be added
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Remove stop words
data_words_nostops = remove_stopwords(data_words)

# Form bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [15]:
print(data_lemmatized[0])

['asian', 'exporter', 'fear', 'damage', 'mount', 'trade', 'friction', 'raise', 'fear', 'many', 'asia', 'export', 'nation', 'row', 'could', 'inflict', 'far', 'reach', 'economic', 'damage', 'businessman', 'official', 'say', 'tell', 'asian', 'capital', 'move', 'may', 'boost', 'protectionist_sentiment', 'lead', 'curb', 'american', 'import', 'product', 'exporter', 'say', 'conflict', 'would', 'hurt', 'long', 'run', 'tokyos', 'loss', 'may', 'gain', 'say', 'impose', 'dlrs', 'import', 'japanese', 'electronic', 'good', 'retaliation', 'stick', 'pact', 'sell', 'semiconductor', 'world', 'market', 'cost', 'unofficial', 'japanese', 'estimate', 'put', 'impact', 'tariff', 'dlrs', 'spokesman', 'major', 'electronic', 'firm', 'say', 'would', 'virtually', 'halt', 'export', 'product', 'hit', 'new', 'taxis', 'would', 'able', 'business', 'say', 'spokesman', 'lead', 'japanese', 'electronic', 'firm', 'tariff', 'remain', 'place', 'length', 'time', 'month', 'mean', 'complete', 'erosion', 'export', 'good', 'tariff

In [16]:
# Create dictionary and corpus for topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 4), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 3), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 2), (35, 1), (36, 3), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 4), (48, 1), (49, 1), (50, 2), (51, 2), (52, 1), (53, 4), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 8), (61, 2), (62, 1), (63, 1), (64, 2), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 6), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 5), (86, 1), (87, 2), (88, 4), (89, 3), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 3), (98, 1), (99, 4), (100, 1), (101, 1), (102, 3), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1),

Note that bigram such as 'domestically_produced' and 'protectionist_sentiment' are included in the dictionary.

In [17]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('accounting', 1),
  ('action', 1),
  ('advantage', 1),
  ('allege', 1),
  ('allow', 2),
  ('also', 4),
  ('american', 1),
  ('asia', 1),
  ('asian', 2),
  ('ask', 1),
  ('australia', 1),
  ('australian', 1),
  ('avow', 1),
  ('await', 1),
  ('aware', 1),
  ('barrier', 1),
  ('beef', 1),
  ('big', 1),
  ('block', 1),
  ('boost', 2),
  ('budget', 1),
  ('business', 1),
  ('businessman', 3),
  ('call', 1),
  ('capital', 1),
  ('centre', 1),
  ('chief', 1),
  ('coal', 1),
  ('commercial', 1),
  ('complete', 1),
  ('concern', 2),
  ('conflict', 1),
  ('continue', 1),
  ('cost', 2),
  ('could', 1),
  ('country', 3),
  ('curb', 1),
  ('cut', 1),
  ('damage', 2),
  ('day', 1),
  ('defuse', 1),
  ('deputy', 1),
  ('deterioration', 1),
  ('diplomatic', 1),
  ('disadvantage', 1),
  ('dispute', 2),
  ('dlrs', 4),
  ('domestically_produced', 1),
  ('due', 1),
  ('economic', 2),
  ('economy', 2),
  ('effort', 1),
  ('electronic', 4),
  ('emergency', 1),
  ('end', 1),
  ('erosion', 

## Topic Modeling Training

### Preliminary Topic Model

We generate a topic model for the article.  We employ two approaches:

1. Gensim's built-in LDA topic modeling function, `ldamodel`.  Gensim implementations of LDA employ Variational Inference, which introduces bias and can provide less coherence in topics.
2. MALLET's LDA topic model with a Gensim wrapper, `ldamallet`.  This model employs Gibbs sampling (an MCMC method), which introduces is unbiased at the cost of heavy computation (performed in Java)

LDA models can be evaluated based on several different metrics:

- **[Perplexity](https://en.wikipedia.org/wiki/Perplexity)** is a measure of 'surprise', or how well the model predicts.  The lower the score, the better the prediction.
- Coherence is the degree of 'semantic similarity between high scoring keywords in the topic' [Learn to Find Topics in a Text Corpus](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0).  It is computed as a the sum of pairwise scores of the top n words by frequency.  The higher the coherence, the more coherent and interpretable the topic. 

Selecting the number of topics in an LDA model can be an arbitrary decision. We inform this decision through hyperparameter tuning based on an evaluation of the topic coherence yielded by different numbers of topics.  We optimize the Gensim model accordingly (JO: MALLET too?)

In [1]:
# Function to iterate over numbers of topics and calculates LDA model coherence
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
   
    coherence_values = []
    model_list = []
    coherence='c_v'
    
    for num_topics in range(start, limit, step):
        
        if model_type == 'LdaModel':
        
            model = gensim.models.ldamodel.LdaModel(corpus=corpus,  # TDF corpus
                                               id2word=id2word,  # dictionary
                                               num_topics=num_topics,  # to be optimized
                                               random_state=100,  # [JO: CONFIRM]
                                               # update_every=1,  # how often paramater should be updated
                                               # chunksize=100,
                                               # passes=10,  # total number of training passes
                                               # alpha='auto',  # learn assymetric alpha from training data (JO: consider whether to evaluate symmetric approach for alpha with asymmetric for beta)
                                               per_word_topics=True)
        
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model, 
                                            texts=texts, 
                                            dictionary=dictionary, 
                                            coherence=coherence)
            coherence_values.append(coherencemodel.get_coherence())

        if model_type == 'LdaMallet':
            
            mallet_path = './mallet-2.0.8/bin/mallet' 
            
            model = gensim.models.wrappers.LdaMallet(mallet_path, 
                                                     corpus=corpus, 
                                                     num_topics=num_topics, 
                                                     id2word=id2word)
            
            model_list.append(model)
            coherencemodel = CoherenceModel(model=ldamallet, 
                                                texts=texts,
                                                dictionary=id2word, 
                                                coherence=coherence)
            cohernece_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values  # List of LDA topic models and coherence values for respective number of topics

# Function to support decision on optimal number of topics
def chart_model_coherence(coherence_values, limit, start=2, step=3):
    
    x = range(start, limit, step)
    
    coherence_table = pd.DataFrame(list(zip(x,coherence_values)), columns=['Num Topics','Coherence Score'])
    
    plt.plot(x, coherence_values)
    plt.xlabel('Num Topics')
    plt.ylabel('Coherence Score')
    plt.legend(('coherence_values'), loc='best')
    plt.show()
    
    return coherence_table

### Gensim LDA model

In [ ]:
# Cast broad net to find neighborhood of optimal number of topics in range of 2 to 40 in steps of 4
gs_broad_model_list, gs_broad_coherence_values = compute_coherence_values(dictionary=id2word, 
                                                                        corpus=corpus, 
                                                                        texts=data_lemmatized, 
                                                                        start=2, 
                                                                        limit=40, 
                                                                        step=4)

# Chart broad net of optimal number of topics
chart_model_coherence(gs_broad_coherence_values, 
                      start=2, 
                      limit=40, 
                      step=4)

When analyzed at 4 unit increments, Gensim LDA model coherence peaks between 6 and 25 topics.  It's not very smooth, so given the spikes we narrow our search in that range to find the optimum.

In [ ]:
# Cast narrower net in range of 6 to 25 in single steps to smooth and find optimum in number of topics
gs_narrow_model_list, gs_narrow_coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        texts=data_lemmatized, 
                                                        start=6, 
                                                        limit=25, 
                                                        step=1)

# Chart narrower net of optimal number of topics
gs_narrow_coherence = chart_model_coherence(gs_narrow_coherence_values, 
                      start=8, 
                      limit=25, 
                      step=1)     
gs_narrow_coherence

9 topics yield the highest coherence score.  We select the Gensim model with 9 topics.

In [ ]:
# Optimal Gensim model
gs_optimal_model = gs_narrow_model_list[int(gs_narrow_coherence.loc[gs_narrow_coherence['Coherence Score'].idxmax()]['Num Topics'])]

# Compute model perplexity and coherence score
print('\nGensim LDA model perplexity: ', lda_model.log_perplexity(corpus))
print('\nGensim LDA model coherence score: ', gs_narrow_coherence.loc[gs_narrow_coherence['Coherence Score'].idxmax()]['Coherence Score'])

In [ ]:
# Here is a quick look at the the top words by topic for the optimal Gensim model
for topic_id in range(gs_optimal_model.num_topics):
    top_k = optimal_model.show_topic(topic_id, 10)
    top_k_words = [w for w, _ in top_k]
    
    print('{}: {}'.format(topic_id, ' '.join(top_k_words)))

### MALLET LDA model

JO: Mael, what if we trained both models first in this section, and then compared their relative performance before selecting MALLET and analyzing its output?

## Topic Model Analysis

### Topic Prevalence and Distribution 

In [26]:
# JO: troubleshoot TypeError:  https://cs50.stackexchange.com/questions/30671/typeerror-not-supported-between-instances-of-int-and-tuple


# MI: this is now included in the best model functions.

# Find dominant topic by sentence
# def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
#     # Init output
#     sent_topics_df = pd.DataFrame()

#     # Loop through articles to get each article's main topic
#     for i, article in enumerate(ldamodel[corpus]):
#         for article in enumerate(article):  # JO: removed the sort operation here - think we'll need to figure out another way to resort the article value, but leaving for now
#             if article[0] == 0:
                
#                 print(article)
                
#                 wp = optimal_model.show_topic(article[1][0][0])
#                 topic_keywords = ', '.join([word for word, prop in wp])
#                 sent_topics_df = sent_topics_df.append(pd.Series([int(article[1][0][0]), 
#                                                                   round(article[1][0][1],4), 
#                                                                   topic_keywords]), 
#                                                        ignore_index=True)
#             else:
#                 break

#     sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

#     return(sent_topics_df)


In [28]:
# df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
# df_dominant_topic = df_topic_sents_keywords.reset_index()
# df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords']

# Show
#df_dominant_topic.head(10)

In [ ]:
# Will need to be adjusted for Gensim rather than MALLET

# Find most representative document by topic
# Group top 5 sentences under each topic
# sent_topics_sorted_df = pd.DataFrame()

# sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

# for i, grp in sent_topics_outdf_grpd:
#     sent_topics_sorted_df = pd.concat([sent_topics_sorted_df, 
#                                              grp.sort_values(['Perc_Contribution'], 
#                                                              ascending=[0]).head(1)], 
#                                             axis=0)

# # Reset Index    
# sent_topics_sorted_df.reset_index(drop=True, inplace=True)

# # Format
# sent_topics_sorted_df.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords"]

# # Show
# sent_topics_sorted_df.head(n=10)

In [ ]:
#sent_topics_sorted_df

In [ ]:
# Depends on preceding code which needs troubleshooting
# Will need to be adjusted for Gensim rather than MALLET

# Evaluate topic distribution across documents

# # Number of documents by topic
# topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# # Percentage of documents by topic
# topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# # Topic number and keywords
# topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# # Concatenate columns
# df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# # Set column names
# df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# df_dominant_topics

In [ ]:
# JO: assuming MALLET is the top performer, do this for MALLET ony?

# Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

## Topic Modeling using MALLET

### TODO: Build Mallet optimal model directly

In [30]:
# Build LDA Mallet model
mallet_path = './mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

# Show topics
pprint(ldamallet.show_topics(formatted=False))

[(7,
  [('company', 0.09249665029030818),
   ('sell', 0.05600714604734256),
   ('unit', 0.05462259937472086),
   ('buy', 0.04917373827601608),
   ('group', 0.041715051362215275),
   ('business', 0.03729343456900402),
   ('firm', 0.028584189370254576),
   ('investment', 0.027020991514068782),
   ('product', 0.018088432335864227),
   ('acquire', 0.0150960250111657)]),
 (19,
  [('spokesman', 0.041512960503664355),
   ('today', 0.021887757611529193),
   ('pay', 0.01844473956027741),
   ('ship', 0.017559392061384092),
   ('strike', 0.015247651369829324),
   ('begin', 0.013968816093650091),
   ('state', 0.01391963012148935),
   ('close', 0.013821258177167872),
   ('work', 0.013772072205007133),
   ('port', 0.011017657764005706)]),
 (18,
  [('rate', 0.14285069710779227),
   ('interest', 0.05937824301764202),
   ('cut', 0.03794612642692776),
   ('week', 0.03343410188151424),
   ('day', 0.022289401254342825),
   ('credit', 0.020755312908902226),
   ('bank', 0.01822857916347065),
   ('point', 0.

In [31]:
# Compute coherence score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, 
                                            texts=data_lemmatized,
                                            dictionary=id2word, 
                                            coherence='c_v')

coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence score: ', coherence_ldamallet)


Coherence score:  0.5022628060961075


### Visualize Optimal Mallet

In [32]:
# Visualize the topics

# MI

# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
# vis

### Dataframes

In [ ]:
# Extract best topic for each article along with associate keywords. 

# MI: Turned this into two functions, one for mallet, one for gensim. This can be removed

# art_topic_keywords = pd.DataFrame(columns = ['article','best_topic','likelyhood','keywords'])

# for i in range(len(lda_mallet_corpus)):
#     article = lda_mallet_corpus[i]
    
#     #print("Article " + str(i))
    
#     # Find the max likelyhood index and select the corresponding topic
#     likelyhood = [likelyhood for topic,likelyhood in article]
#     max_likelyhood = max(likelyhood)
#     best_topic = likelyhood.index(max_likelyhood)
    
#     #print('most likely topic: {}, likelyhood: {}'.format(best_topic, max_likelyhood))
    
#     wp = ldamallet.show_topic(best_topic)
#     #pprint(wp)
#     topic_keywords = ', '.join([word for word, prop in wp])
#     #print(topic_keywords)
#     d = {'article': i, 'best_topic': best_topic,'likelyhood': max_likelyhood,'keywords': topic_keywords}
#     art_topic_keywords = art_topic_keywords.append(d, ignore_index=True)

### Best Gensim Topics

In [34]:
# Extract best topic for each article along with associate keywords and likelyhood
def get_best_gensim_topics(lda_model, corpus):
    
    lda_model_corpus = lda_model[corpus]
    art_topic_keywords = pd.DataFrame(columns = ['article','best_topic','likelyhood','keywords'])
    
    # Loop through articles to get each article's main topic
    for i, article in enumerate(lda_model[corpus]):
        for article in enumerate(article): 
            if article[0] == 0:
                #print(article)
                wp = optimal_model.show_topic(article[1][0][0])
                topic_keywords = ', '.join([word for word, prop in wp])
                
                best_topic = int(article[1][0][0])
                max_likelyhood = round(article[1][0][1],4)
                
                d = {'article': i, 'best_topic': best_topic,'likelyhood': max_likelyhood,'keywords': topic_keywords}
                art_topic_keywords = art_topic_keywords.append(d, ignore_index=True)

    return art_topic_keywords

In [35]:
lda_gensim_opt_model = optimal_model
gensim_topic_df = get_best_gensim_topics(lda_gensim_opt_model, corpus)

In [36]:
gensim_topic_df

,article,best_topic,likelyhood,keywords
0,0,6,0.7154,"say, trade, would, export, official, country, ..."
1,1,3,0.1170,"say, company, share, offer, would, stock, buy,..."
2,2,2,0.0152,"say, soybean, acre, program, regular, acreage,..."
3,3,0,0.1791,"say, rate, growth, economy, would, economic, d..."
4,4,4,0.0337,"dollar, market, bank, dealer, yen, money, say,..."
...,...,...,...,...
10783,10783,4,0.9260,"dollar, market, bank, dealer, yen, money, say,..."
10784,10784,20,0.7596,"ct, record, dividend, stock, split, share, may..."
10785,10785,20,0.7596,"ct, record, dividend, stock, split, share, may..."
10786,10786,18,0.8886,"loss, ct, net, profit, dlrs, year, note, inclu..."


### Best Mallet Topics

In [37]:
# Extract best topic for each article along with associate keywords and likelyhood
def get_best_mallet_topics(lda_model, corpus):
    lda_model_corpus = lda_model[corpus]
    art_topic_keywords = pd.DataFrame(columns = ['article','best_topic','likelyhood','keywords'])
    
    for i in range(len(lda_model_corpus)):
        # Get article topic model distribution
        article = lda_model_corpus[i]

        # Find the max likelyhood index and select the corresponding topic
        likelyhood = [likelyhood for topic,likelyhood in article]
        max_likelyhood = max(likelyhood)
        best_topic = likelyhood.index(max_likelyhood)
        
        # Get keywords for best topcis
        wp = lda_model.show_topic(best_topic)
        topic_keywords = ', '.join([word for word, prop in wp])
        
        # Assemble and append information to dataframe
        d = {'article': i, 'best_topic': best_topic,'likelyhood': max_likelyhood,'keywords': topic_keywords}
        art_topic_keywords = art_topic_keywords.append(d, ignore_index=True)
        
    return art_topic_keywords
    

In [38]:
mallet_topic_df = get_best_mallet_topics(ldamallet, corpus)

In [39]:
mallet_topic_df

,article,best_topic,likelyhood,keywords
0,0,10,0.258945,"official, japanese, government, source, indust..."
1,1,12,0.121967,"growth, government, economic, year, economy, d..."
2,2,17,0.231944,"oil, price, production, crude, barrel, increas..."
3,3,15,0.199495,"year, export, month, end, import, record, tota..."
4,4,17,0.198211,"oil, price, production, crude, barrel, increas..."
...,...,...,...,...
10783,10783,3,0.128704,"dollar, bank, market, exchange, currency, fore..."
10784,10784,16,0.084906,"ct, loss, net, profit, note, include, gain, dl..."
10785,10785,16,0.084906,"ct, loss, net, profit, note, include, gain, dl..."
10786,10786,16,0.263889,"ct, loss, net, profit, note, include, gain, dl..."


## Topic Comparison

Discuss overlap between columns. Maybe read an example article to validate. Talk about verification.

In [40]:
# Create summary dataframe with keywords from each model vs known categories
summary_df = pd.concat([df['categories'], gensim_topic_df['keywords'], mallet_topic_df['keywords']], axis=1, sort=False)
summary_df.columns = ['reuters_categories','gensim_keywords','mallet_keywords']

In [41]:
summary_df

,reuters_categories,gensim_keywords,mallet_keywords
0,[trade],"say, trade, would, export, official, country, ...","official, japanese, government, source, indust..."
1,[grain],"say, company, share, offer, would, stock, buy,...","growth, government, economic, year, economy, d..."
2,"[crude, nat-gas]","say, soybean, acre, program, regular, acreage,...","oil, price, production, crude, barrel, increas..."
3,"[corn, grain, rice, rubber, sugar, tin, trade]","say, rate, growth, economy, would, economic, d...","year, export, month, end, import, record, tota..."
4,"[palm-oil, veg-oil]","dollar, market, bank, dealer, yen, money, say,...","oil, price, production, crude, barrel, increas..."
...,...,...,...
10783,"[interest, money-fx]","dollar, market, bank, dealer, yen, money, say,...","dollar, bank, market, exchange, currency, fore..."
10784,[earn],"ct, record, dividend, stock, split, share, may...","ct, loss, net, profit, note, include, gain, dl..."
10785,[earn],"ct, record, dividend, stock, split, share, may...","ct, loss, net, profit, note, include, gain, dl..."
10786,[earn],"loss, ct, net, profit, dlrs, year, note, inclu...","ct, loss, net, profit, note, include, gain, dl..."


In [42]:
# Show example 

In [43]:
# Print chosen dataframe to csv file
mallet_topic_df.to_csv('mallet_df.csv', index=False) 

## Network Analysis

- Expecting a disconnected graph with num_topics central nodes with edges to articles. 
- Get the top 5 largest subgraphs which represent the most common topics (degree centrality)
- From the most popular topics, suggest next 5 articles to read by looking at edge weight (likelyhood).

In [44]:
# Read dataframe from csv file and build a graph from it.

In [45]:
import networkx as nx
import networkx.algorithms.bipartite as bipartite
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [46]:
G_full = nx.from_pandas_edgelist(art_topic_keywords,'article','best_topic')
G_full.graph['name'] = 'Full bipartite graph'
print(nx.info(G_full))

NameError: name 'art_topic_keywords' is not defined

In [ ]:
s = 'Graph is bipartite: {} \nGraph is connected: {} \nNumber of connected components {}'
print(s.format(nx.is_bipartite(G_full), nx.is_connected(G_full), nx.number_connected_components(G_full)))

In [ ]:
# Identify connected components with greater than five nodes
sorted([len(c) for c in nx.connected_components(G_full) if len(c) > 5], reverse=True)

In [ ]:
# Function to extract subgraphs
def get_subgraphs(graph):
    
    subgraphs = [(graph.subgraph(c),len(c)) for c in nx.connected_components(graph) if len(c) > 5] # networkx 2.4
    return sorted(subgraphs, key = lambda x: x[1], reverse=True)

# Create connected subgraphs and confirm
subgraphs = get_subgraphs(G_full)
print(*subgraphs[:5], '\n', sep='\n')

# Isolate the largest subgraph
largest_subg = subgraphs[0][0]
largest_subg.graph['name'] = 'Main bipartite subgraph'
print(nx.info(largest_subg), "\n")

# Verify largest subgraph is bipartite
G = largest_subg
print('Graph is bipartite: {} \nGraph is connected: {} \n'.format(nx.is_bipartite(G), nx.is_connected(G)))

In [ ]:
# Source node labels
graph = G
articles, topics = nx.bipartite.sets(graph)

# Plot graph by node type
# Apply plot settings
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (24,12)
plt.axis('off')
pos = nx.spring_layout(graph)
nx.draw_networkx_nodes(graph, pos, nodelist=articles, node_color='red', alpha = 0.4)
nx.draw_networkx_nodes(graph, pos, nodelist=topics, node_color='blue', alpha = 0.4, node_size = 1000)
nx.draw_networkx_edges(graph, pos, alpha = 0.4)
nx.draw_networkx_labels(graph, pos);

## Conclusion

## YouTube

## References

(JO: add remaining references)

**Topic Modeling**

https://en.wikipedia.org/wiki/Topic_model
https://springerplus.springeropen.com/articles/10.1186/s40064-016-3252-8
https://mimno.infosci.cornell.edu/papers/2017_fntir_tm_applications.pdf
https://cfss.uchicago.edu/notes/topic-modeling/

**LDA**
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation
https://en.wikipedia.org/wiki/Dirichlet_distribution https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158

**Perplexity and Coherence**
https://en.wikipedia.org/wiki/Perplexity
https://cfss.uchicago.edu/notes/topic-modeling/ (provides examples on optimizing perplexity
https://mimno.infosci.cornell.edu/info6150/readings/N10-1012.pdf (automatic topic coherence evaluation)
https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
http://qpleple.com/topic-coherence-to-evaluate-topic-models/ (intrinsic / extrinsic measures and associated math)

**Gensim**
https://radimrehurek.com/gensim/auto_examples/index.html
https://www.thinkinfi.com/2019/08/LDA-Gensim-Python.html
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
https://towardsdatascience.com/building-a-topic-modeling-pipeline-with-spacy-and-gensim-c5dc03ffc619

**Spacy**
https://spacy.io/

**MALLET**
http://mallet.cs.umass.edu/topics.php
https://www.thinkinfi.com/2019/08/LDA-Gensim-Python.html

**Text Network Analysis**
https://advances.sciencemag.org/content/4/7/eaaq1360
https://noduslabs.com/wp-content/uploads/2019/06/InfraNodus-Paranyushkin-WWW19-Conference.pdf
https://noduslabs.com/cases/tutorial-lda-text-mining-network-analysis/
https://github.com/michal-pikusa/text-network-analysis
https://github.com/martingerlach/hSBM_Topicmodel/blob/master/TopSBM-tutorial.ipynb